# 1. import modules

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
%matplotlib inline
import mdtraj as md
import nglview as nv
import dmaps
import math
from scipy.spatial.distance import pdist, squareform
from mpl_toolkits.mplot3d import Axes3D
from numpy import matlib as mb

_ColormakerRegistry()

# 2. gyration tensor calculation

In [2]:
traj = md.load('trj_whole.xtc', top='traj_pbc.gro')

In [3]:
traj_20000 = traj[:20000]

In [4]:
def principal_ax(traj):
    traj.center_coordinates()
    X = np.array([X.T.dot(X)/X.shape[0] for X in traj.xyz])
    evecs = np.linalg.eigvalsh(X)[:,::-1]
    return np.sqrt(evecs)*10

In [5]:
gt = principal_ax(traj)

In [6]:
gt.shape

(50001, 3)

In [7]:
gt1 = gt[:,0]
gt2 = gt[:,1]
gt3 = gt[:,2]

# 3. Takens' delay embedding

In [8]:
dist = np.loadtxt('h2t series.txt', delimiter=',')
length = len(dist)

In [9]:
#tao = 2
#d = 20

In [10]:
# This code conduct delay embedding, given a time series O, delay time tau and
# delay dimension D, it create a D dimensional vectors EBD, and a pairwise
# distance matrix of the delayed points.

def delayembedding(dist,gt):

    d = int(input('Enter the dimension of embeding space:')) # determin dimension of embeding space
    tao = int(input('Enter the delay time T:')) # input delay time
    
    N = traj.xyz.shape[0]
    L = N - d*tao
    M = np.zeros((L,d+1))
    
    gt1M = np.zeros((L,d+1)) 
    gt2M = np.zeros((L,d+1)) 
    gt3M = np.zeros((L,d+1)) 
    
    print('The original number of snapshots is %d, the nunmber of reconstructed points is %d\n'%(length, L)) #show change of # of data points
    
    M[:,0]= np.transpose(dist[0:L])
    for i in range(L):
        for j in range(1,d+1,1):
            M[i,j] = dist[i+j*tao]
            gt1M[i,j] = gt1[i+j*tao]
            gt2M[i,j] = gt2[i+j*tao]
            gt3M[i,j] = gt3[i+j*tao]
    
    return M,gt1M,gt2M,gt3M


In [11]:
M, gt1M, gt2M, gt3M = delayembedding(dist,gt)

Enter the dimension of embeding space:19
Enter the delay time T:2
The original number of snapshots is 50001, the nunmber of reconstructed points is 49963



In [12]:
print(gt1M.shape,gt2M.shape,gt3M.shape,M.shape)

(49963, 20) (49963, 20) (49963, 20) (49963, 20)


In [13]:
np.savetxt('EBD.txt', M)


In [14]:
gt1 = np.zeros(gt1M.shape[0])
gt2 = np.zeros(gt2M.shape[0])
gt3 = np.zeros(gt3M.shape[0])

In [15]:
for i in range(gt1M.shape[0]):
    gt1[i] = np.mean(gt1M[i,:])
    gt2[i] = np.mean(gt2M[i,:])
    gt3[i] = np.mean(gt3M[i,:])

In [16]:
np.savetxt('gt1ebd.txt', gt1)
np.savetxt('gt2ebd.txt', gt2)
np.savetxt('gt3ebd.txt', gt3)